In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [ ]:
import os
os.chdir('/content/gdrive/MyDrive/Colab Notebooks/Finnhub')

In [ ]:

import pandas as pd
import numpy as np
import random
import time
import pickle
import json
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import RobustScaler
from Statistics import Statistics

from keras.models import Sequential
from keras.layers import Dense, LSTM, BatchNormalization
from keras.optimizers import Adam
from keras.callbacks import ReduceLROnPlateau, EarlyStopping
from keras.regularizers import l2
from time import time
import warnings
warnings.filterwarnings("ignore")

import os
SEED = 9
os.environ['PYTHONHASHSEED']=str(SEED)
random.seed(SEED)
np.random.seed(SEED)

In [ ]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [ ]:
def extract_names_from_json(json_file_path):
    with open(json_file_path, "r") as f:
        json_data = json.load(f)

    names = []
    for entry in json_data:
        names.append(entry["Symbol"])

    return names

def reshaper(arr):
    arr = np.array(np.split(arr,3,axis=1))
    arr = np.swapaxes(arr,0,1)
    arr = np.swapaxes(arr,1,2)
    return arr

def simulate(test_data,predictions):
    rets = pd.DataFrame([],columns=['Long','Short'])
    k = 10
    for day in sorted(predictions.keys()):
        preds = predictions[day]
        test_returns = test_data[test_data[:,0]==day][:,-2]
        top_preds = predictions[day].argsort()[-k:][::-1]
        trans_long = test_returns[top_preds]
        worst_preds = predictions[day].argsort()[:k][::-1]
        trans_short = -test_returns[worst_preds]
        rets.loc[day] = [np.mean(trans_long),np.mean(trans_short)]
    print('Result : ',rets.mean())
    return rets

def simulate(test_data,predictions):
    rets = pd.DataFrame([],columns=['Long','Short'])
    k = 10
    for day in sorted(predictions.keys()):
        preds = predictions[day]
        test_returns = test_data[test_data[:,0]==day][:,-2]
        top_preds = predictions[day].argsort()[-k:][::-1]
        trans_long = test_returns[top_preds]
        worst_preds = predictions[day].argsort()[:k][::-1]
        trans_short = -test_returns[worst_preds]
        rets.loc[day] = [np.mean(trans_long),np.mean(trans_short)]
    return rets

def make_predictions(test_data,model):
  predictions = {}

  dates = list(set(test_data[:,0]))
  predictions = {}
  for day in dates:
    test_d = test_data[test_data[:,0]==day]
    test_d = reshaper(test_d[:,2:-2]).astype('float16')
    predictions[day] = model.predict(test_d)[:,1]
  return predictions

def create_label(df_open,df_close,perc=[0.5,0.5]):
    if not np.all(df_close.iloc[:,0]==df_open.iloc[:,0]):
        print('Date Index issue')
        return
    perc = [0.]+list(np.cumsum(perc))
    label = (df_close.iloc[:,1:]/df_open.iloc[:,1:]-1).apply(
            lambda x: pd.qcut(x.rank(method='first'),perc,labels=False), axis=1)
    return label[1:]

def create_stock_data(df_close,df_open,st,label,test_year,m=240):
    st_data = pd.DataFrame([])
    st_data['Date'] = list(df_close['Date'])
    st_data['Name'] = [st]*len(st_data)
    daily_change = df_close[st]/df_open[st]-1
    for k in range(m)[::-1]:
        st_data['IntraR'+str(k)] = daily_change.shift(k)
    nextday_ret = (np.array(df_open[st][1:])/np.array(df_close[st][:-1])-1)
    nextday_ret = pd.Series(list(nextday_ret)+[np.nan])
    for k in range(m)[::-1]:
        st_data['NextR'+str(k)] = nextday_ret.shift(k)
    close_change = df_close[st].pct_change()
    for k in range(m)[::-1]:
        st_data['CloseR'+str(k)] = close_change.shift(k)
    st_data['IntraR-future'] = daily_change.shift(-1)
    st_data['label'] = list(label[st])+[np.nan]
    st_data['Month'] = list(df_close['Date'].str[:-3])
    st_data = st_data.dropna()

    trade_year = st_data['Month'].str[:4]
    st_data = st_data.drop(columns=['Month'])
    st_train_data = st_data[trade_year<str(test_year)]
    st_test_data = st_data[trade_year==str(test_year)]
    return np.array(st_train_data),np.array(st_test_data)

def scalar_normalize(train_data,test_data):
    scaler = RobustScaler()
    scaler.fit(train_data[:,2:-2])
    train_data[:,2:-2] = scaler.transform(train_data[:,2:-2])
    test_data[:,2:-2] = scaler.transform(test_data[:,2:-2])

def calculate_accuracy(predictions, test_data):
    accuracies = {}

    for date, daily_predictions in predictions.items():
        filtered_data = test_data[test_data[:, 0] == date]

        # Convert predictions to binary labels
        binary_predictions = [1 if prediction >= 0.5 else 0 for prediction in daily_predictions]

        # Calculate accuracy for the day
        correct_predictions = 0
        for prediction, label in zip(binary_predictions, filtered_data[:, -1]):
            if prediction == label:
                correct_predictions += 1

        accuracy = correct_predictions / len(binary_predictions)
        accuracies[date] = accuracy

    accuracy = np.mean(list(accuracies.values()))

    return accuracy

model_folder = 'models-Intraday-240-3-LSTM_RNN'
result_folder = 'results-Intraday-240-3-LSTM_RNN'
for directory in [model_folder,result_folder]:
    if not os.path.exists(directory):
        os.makedirs(directory)


In [ ]:

json_file_path = "s&p500.json"
names = extract_names_from_json(json_file_path)

result_folder = 'results-Intraday-240-3-LSTM_RNN'
for directory in [result_folder]:
    if not os.path.exists(directory):
        os.makedirs(directory)

In [ ]:
filename = '/content/gdrive/MyDrive/Colab Notebooks/Finnhub/Open-1996.xlsx'
df_open = pd.read_excel(filename)
filename = '/content/gdrive/MyDrive/Colab Notebooks/Finnhub/Close-1996.xlsx'
df_close = pd.read_excel(filename)

# print(df_open.drop(df_open.columns[df_open.isnull().any()], axis=1, inplace=True))
label = create_label(df_open,df_close)

In [ ]:
constituents = {}

for index, row in df_open.iterrows():
  date = row['Date']
  stocks = list(row.dropna().index.drop('Date'))
  constituents[date] = stocks

In [ ]:
for test_year in range(2014,2023):

  print('-'*40)
  print(test_year)
  print('-'*40)

  df_close_cur = df_close[(df_close['Date'].str[:4].astype(int) >= test_year -3) & (df_close['Date'].str[:4].astype(int) <= test_year)].reset_index(drop=True)
  df_open_cur = df_open[(df_open['Date'].str[:4].astype(int) >= test_year - 3) & (df_open['Date'].str[:4].astype(int) <= test_year)].reset_index(drop=True)

  # print(df_open_cur.head())
  label = create_label(df_open_cur,df_close_cur)

  earliest_date = ''

  for key in constituents:
      if key.startswith(str(test_year-3)):
          if earliest_date == '':
            earliest_date = key
          elif key < earliest_date:
            earliest_date = key

  stock_names = sorted(list(constituents[earliest_date]))

  train_data,test_data = [],[]

  for st in stock_names:
    st_train_data,st_test_data = create_stock_data(df_close_cur,df_open_cur,st,label,test_year)
    train_data.append(st_train_data)
    test_data.append(st_test_data)

  train_data = np.concatenate([x for x in train_data])
  test_data = np.concatenate([x for x in test_data])
  scalar_normalize(train_data,test_data)

  np.random.shuffle(train_data)
  train_x,train_y,train_ret = train_data[:,2:-2],train_data[:,-1],train_data[:,-2]
  train_x = train_x.astype('float16')
  train_y = train_y.astype('float16')
  train_x = reshaper(train_x)
  train_y = np.reshape(train_y,(-1, 1))
  enc = OneHotEncoder(handle_unknown='ignore')
  enc.fit(train_y)
  enc_y = enc.transform(train_y).toarray()

  cutoff_index = np.argmax(test_data[:,0] >= (str(test_year) + '-05-01'))
  validation_data = test_data[:cutoff_index]
  validation_x,validation_y,validation_ret = validation_data[:,2:-2],validation_data[:,-1],validation_data[:,-2]
  validation_x = validation_x.astype('float16')
  validation_y = validation_y.astype('float16')
  validation_x = reshaper(validation_x)
  validation_y = np.reshape(validation_y,(-1, 1))
  enc = OneHotEncoder(handle_unknown='ignore')
  enc.fit(validation_y)
  validation_y = enc.transform(validation_y).toarray()

  # Let's make a list of CONSTANTS for modelling:
  LAYERS = [4, 4, 2]                # number of units in hidden and output layers
  M_TRAIN = train_x.shape[0]           # number of training examples (2D)
  M_TEST = validation_x.shape[0]             # number of test examples (2D),full=X_test.shape[0]
  N = train_x.shape[2]                 # number of features
  BATCH = M_TRAIN                          # batch size
  EPOCH = 50                           # number of epochs
  LR = 5e-2                            # learning rate of the gradient descent
  LAMBD = 3e-2                         # lambda in L2 regularizaion
  DP = 0.0                             # dropout rate
  RDP = 0.0                            # recurrent dropout rate
  T = 240
  print(f'layers={LAYERS}, train_examples={M_TRAIN}, test_examples={M_TEST}')
  print(f'batch = {BATCH}, timesteps = {T}, features = {N}, epochs = {EPOCH}')
  print(f'lr = {LR}, lambda = {LAMBD}, dropout = {DP}, recurr_dropout = {RDP}')

  # Build the Model
  model = Sequential()
  model.add(LSTM(input_shape=(T, N), units=LAYERS[0],
                activation='tanh', recurrent_activation='hard_sigmoid',
                kernel_regularizer=l2(LAMBD), recurrent_regularizer=l2(LAMBD),
                dropout=DP, recurrent_dropout=RDP,
                return_sequences=True, return_state=False,
                stateful=False, unroll=False
                ))
  model.add(BatchNormalization())
  # model.add(LSTM(units=LAYERS[1],
  #               activation='tanh', recurrent_activation='hard_sigmoid',
  #               kernel_regularizer=l2(LAMBD), recurrent_regularizer=l2(LAMBD),
  #               dropout=DP, recurrent_dropout=RDP,
  #               return_sequences=True, return_state=False,
  #               stateful=False, unroll=False
  #               ))
  # model.add(BatchNormalization())
  model.add(LSTM(units=LAYERS[1],
                activation='tanh', recurrent_activation='hard_sigmoid',
                kernel_regularizer=l2(LAMBD), recurrent_regularizer=l2(LAMBD),
                dropout=DP, recurrent_dropout=RDP,
                return_sequences=False, return_state=False,
                stateful=False, unroll=False
                ))
  model.add(BatchNormalization())
  model.add(Dense(units=LAYERS[2], activation='sigmoid'))

  # Compile the model with Adam optimizer
  model.compile(loss='binary_crossentropy',
                metrics=['accuracy'],
                optimizer=Adam(lr=LR))
  print(model.summary())

  # Define a learning rate decay method:
  lr_decay = ReduceLROnPlateau(monitor='loss',
                              patience=1, verbose=0,
                              factor=0.5, min_lr=1e-8)
  # Define Early Stopping:
  early_stop = EarlyStopping(monitor='loss', min_delta=0,
                            patience=30, verbose=1, mode='auto',
                            baseline=0, restore_best_weights=True)
  # Train the model.
  # The dataset is small for NN - let's use test_data for validation
  start = time()
  History = model.fit(train_x, enc_y,
                      epochs=EPOCH,
                      batch_size=BATCH,
                      validation_split=0.2,
                      validation_data=(validation_x, validation_y),
                      shuffle=True,verbose=0,
                      callbacks=[lr_decay, early_stop])
  print('-'*65)
  print(f'Training was completed in {time() - start:.2f} secs')
  print('-'*65)


  predictions = make_predictions(test_data,model)
  accuracy = calculate_accuracy(predictions, test_data)
  returns = simulate(test_data,predictions)
  returns.to_csv(result_folder+'/avg_daily_rets-'+str(test_year)+'.csv')

  result = Statistics(returns.sum(axis=1))
  print('\nAverage returns prior to transaction charges')
  result.report()
  with open(result_folder+"/report-" + str(test_year) + ".txt", "a") as f:
    res = '-'*30 + '\n'
    res += str(test_year) + '\n'
    res += 'Mean = ' + str(result.mean()) + '\n'
    res += 'Standard dev = '+str(result.std()) + '\n'
    res += 'Sharpe ratio = ' + str(result.sharpe()) + '\n'
    res += 'Standard Error = '+str(result.stderr()) + '\n'
    res += 'Share>0 = ' + str(result.pos_perc()) + '\n'
    res += 'Skewness = '+str(result.skewness()) + '\n'
    res += 'Kurtosis = ' + str(result.kurtosis()) + '\n'
    res += 'VaR_1 = '+str(result.VaR(1)) + '\n'
    res += 'VaR_2 = ' + str(result.VaR(2)) + '\n'
    res += 'VaR_5 = '+str(result.VaR(5)) + '\n'
    res += 'CVaR_1 = ' + str(result.CVaR(1)) + '\n'
    res += 'CVaR_2 = '+str(result.CVaR(2)) + '\n'
    res += 'CVaR_5 = '+str(result.CVaR(5)) + '\n'
    res += 'MDD = '+str(result.MDD()) + '\n'
    res += 'Percentiles = '+str(result.percentiles()) + '\n'
    res += '-'*30 + '\n'
    f.write(res)
  # with open(result_folder+"/avg_returns" + str(test_year) + '.txt', "a") as myfile:
  #     res = '-'*30 + '\n'
  #     res += str(test_year) + '\n'
  #     res += 'Mean = ' + str(result.mean()) + '\n'
  #     res += 'Sharpe = '+str(result.sharpe()) + '\n'
  #     res += 'Accuracy = ' + str(accuracy) + '\n'
  #     res += '-'*30 + '\n'
  #     myfile.write(res)

----------------------------------------
2014
----------------------------------------


layers=[4, 4, 2], train_examples=229951, test_examples=83
batch = 229951, timesteps = 240, features = 3, epochs = 50
lr = 0.05, lambda = 0.03, dropout = 0.0, recurr_dropout = 0.0


Model: "sequential_15"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_30 (LSTM)              (None, 240, 4)            128       
                                                                 
 batch_normalization_30 (Ba  (None, 240, 4)            16        
 tchNormalization)                                               
                                                                 
 lstm_31 (LSTM)              (None, 4)                 144       
                                                                 
 batch_normalization_31 (Ba  (None, 4)                 16        
 tchNormalization)                                               
                                                                 
 dense_15 (Dense)            (None, 2)                 10        
                                                                 
Total params: 314 (1.23 KB)
Trainable params: 298 (1.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-10-aedfc814c364>", line 132, in <cell line: 1>
    returns.to_csv(result_folder+'/avg_daily_rets-'+str(test_year)+'.csv')
  File "/usr/local/lib/python3.10/dist-packages/pandas/util/_decorators.py", line 211, in wrapper
    return func(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/pandas/core/generic.py", line 3720, in to_csv
    return DataFrameRenderer(formatter).to_csv(
  File "/usr/local/lib/python3.10/dist-packages/pandas/util/_decorators.py", line 211, in wrapper
    return func(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py", line 1189, in to_csv
    csv_formatter.save()
  File "/usr/local/lib/python3.10/dist-packages/pandas/io/formats/csvs.py", line 241, in save
    with get_handle(
  File "/usr/loca

In [ ]:
predictions = make_predictions(test_data,model)

16/16 [==============================] - 1s 85ms/step
